<a href="https://colab.research.google.com/github/natasseo/likelionAI/blob/master/Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1강 과제

## 언어 분야 인공지능 기술
* BERT를 활용한 챗봇 시스템
  
  [원문 보기](https://blog.est.ai/2019/11/task-oriented-dialog-systems-meet-bert/)

  __*BERT란 무엇인가*__
    
  2018년 Google은 BERT(Bidirectional Encoder Representations from Transformers)라는 모델을 선보였다. 이 모델은 11개의 NLP Tasks의 성능을 바탕으로 모델들의 순위를 매기는 GLUE(General Language Understanding Evaluation) Benchmark에서 다른 모델들을 큰 차이로 앞서며 최고 성능을 보여주었다. 또한 가장 인간다운 자연어 이해를 기반으로 한 자연어 질문에 대한 답변을 해야하는 태스크 SQUAD(The Stanford Question Answering Dataset) Benchmark에서 기존 최고 성능을 깨면서 주목받기 시작하였다. 실제로 이때부터 거의 모든 NLP Benchmarks에서 BERT를 이용하거나, 개선한 모델들(XLNet, RoBERTa, ALBERT,...)이 상위권을 점유하기 시작하였다.

   BERT는 Transformer를 기반으로 Sentence Embedding 혹은 Contextual Word Embedding을 구하는 네트워크로, 문장을 토큰으로 쪼개서 네트워크에 넣으면 전체 문장에 대한 Vector와 문장 안의 각 단어(정확히 말하면 토큰) 각각에 대응되는 Vector를 출력한다. 이들을 기반으로 Text Classification 등의 Task를 학습하여 수행하면 매우 쉽게 뛰어난 성능을 얻을 수 있는데, 이는 전체 네트워크가 매우 많은 양의 문서로 Masked Language Models을 미리 학습(pre-training)하였기 때문이다.

  __*챗봇에서의 머신러닝 활용*__

  챗봇(Dialog Systems)도 BERT의 바람을 피해가지는 못하였다. 여기서는 특히, 인공지능 스피커에서 주로 이용되는 기능수행 챗봇(Task-Oriented Dialog Systems)중에서 Dialog State Tracking에 대해서만 알아보기로 한다. 이 시스템은 사용자가 얘기한 문장이 어떤 의도인지 알아내는 Intent Classification과 그 문장이 구체적으로 어떤 요청인지 알아내는 Slot Filling으로 구성되어 있다. 예를 들어 사용자가 "예술의 전당까지 가는 택시 좀 불러줘"라고 말한다면, 챗봇은 이 문장의 의도가 "택시호출"이라는 것과 "목적지(slot)=예술의 전당(value)"임을 알아야 택시 호출 API를 통해 정확한 서비스콜을 할 수 있는 것이다.

  기존의 시스템은 이를 인식하기 위해 각각의 문장 분류(Classification) 모델을 학습하였는데, 즉, 어떤 문장이 <택시호출, 목적지, 예술의 전당> 쌍에 해당하는지를 각각 따로따로 학습하는 것이다. 만약 강남구의 레스토랑을 예약하는 문장을 인식하고 싶다면, <레스토랑-예약, 지역, 강남구>에 해당하는 분류 모델을 학습하여야 한다.

  여기서 몇가지 문제가 생기는데, 하나는 챗봇이 어떤 유형의 문장들을 인식하여야 하는지, 즉, <택시호출, 목적지, 예술의 전당>, <레스토랑-예약,지역,강남구> 등을 미리 정의해놓아야 한다는 것이다. 이를 정의한 문서를  Ontology라고 하는데, 문제는 이것이 서비스 중에 업데이트 될 수 있다는 것이다. 원래는 서울 이외의 지역을 가지 않기로 한 택시호출 서비스가 어느날 경기도까지 서비스하기로 결정하였다면, <택시호출, 목적지, 정자역>에 해당하는 문장도 인식해야한다. 근데 그러려면 이에 해당하는 문장을 다시 준비를 해서 학습을 해야하기때문에 업데이트를 하는데 시간이 꽤 걸리게 된다. 더욱이 목적지에 해당하는 값은 매우 많으므로 (예술의 전당, 정자역, 시청, 용산역, 검찰청, …) 각각의 조합에 각각의 분류 모델을 따로 학습해서 서비스를 하는 것은 각각의 데이터를 따로 구해야 하는 어려움이 있으며, 학습/추론 시간도 매우 오래 걸릴수가 있다. 즉, 기존의 시스템은 확장성(scalability)이 떨어진다고 할 수 있다.

  __*BERT를 활용한 확장성있는 챗봇 만들기*__

  CMU의 연구자들은 이를 타개하기 위해 BERT를 Dialog State Tracking에 도입하였다. 이는 BERT 활용의 한 사례인 Span Prediction을 이용하였다. 문장을 BERT에 넣고 문장 전체의 의미를 담고 있는 첫번째 Vector를 이용해서 각 Slot이 해당사항이 없는지(None), 어떤 값이든 상관 안하는지(Don’t Care), 사용자의 얘기에 Slot Value가 존재하는지(Span)를 예측한 후에, 각 토큰의 해당하는 Vector들을 이용하여 Slot Value의 시작점에 대한 확률과 종료점에 대한 확률을 계산하고 이를 바탕으로 Slot Value를 추측하게 된다. 이렇게 되면 최소한 Slot의 Value들을 미리 알고 있지 않아도 된다. 즉, <택시호출, 목적지, 예술의 전당>에서 “예술의전당”이라는 목적지는 미리 알지 못해도 사용자의 얘기에서 추출할 수가 있으므로, <택시호출, 목적지>에 해당하는 모듈만 따로 학습해놓으면 된다. 이는 서울 지역만 가는 택시호출 서비스가 설령 서울 이외의 지역으로 가게 되었다 하더라도, 새로 시스템을 업데이트할 필요가 없다는 것을 뜻한다. 이 모델은 Slot Value들을 미리 알지 못한 상태에서 동작함에도 불구하고, WOZ 2.0 데이터셋에서 최고 성능에 가까운 성능을 얻어서 BERT의 강력함을 보여주었다. 

  __*처음보는 서비스도 처리할 수 있는 챗봇*__

  여기서 한발 더 나아가, BERT의 언어처리 능력을 활용하면 아예 처음보는 서비스나 Slot의 경우에도 처리할 수 있는 Zero-shot Learning이 가능하다. 특히 최근 광범위하게 퍼지고 있는 Voice Assistants(인공지능 스피커)의 경우에는 수많은 서비스들이 꾸준히 추가되고 있는데, 이때마다 다시 학습이 필요하다면 매우 번거로운 일이 될 것이다. 하지만 실상 비슷한 의미의 Slot이 추가되는 경우, 인식하는 방식이 비슷함에도 불구하고 기존의 시스템에서는 따로 학습을 해야했다. 예를들면 비행기 예약 서비스에서의 “예약 시간”과 기차 예약 서비스에서의 “예약 시간”을 인식하는 시스템은 거의 같은 의미임에도 불구하고, 서비스가 다르기 때문에 따로 학습을 해야했었다. 그치만 Slot의 설명(Schema)이 주어진다면 BERT의 언어처리 능력을 통해 이것이 기존의 Slot과 비슷하다는 것을 시스템이 인지할 수 있으므로, 기존에 학습된 모델을 활용해서 새로운 데이터 없이도 (완벽하지는 않지만) 처리가 가능할 것이다. 

## 음성 분야 인공지능 기술
* 가전관리 'LG 씽큐' 앱, AI 음성인식 플랫폼 구글 어시스턴트 탑재
  
  [원문 보기](http://www.aitimes.kr/news/articleView.html?idxno=14088)

 __*국내 가전업체 중 처음으로 가전관리 앱에 음성인식 기능 추가*__

  LG전자가 스마트폰으로 가전제품을 제어하는 ‘LG 씽큐(LG ThinQ)’ 앱(App)에 음성인식 기능을 추가해 고객들이 LG 씽큐 앱을 보다 편리하게 사용할 수 있도록 내달 중 구글 어시스턴트 기반의 음성인식 기능을 탑재하기로 했다. 올해 말까지 무선인터넷(Wi-Fi)이 탑재된 생활가전 全 제품이 스마트폰을 통해 음성으로 제어하는 게 가능해진다. 국내 가전업체 가운데 가전관리 앱에 음성인식 기능을 추가한 것은 LG전자가 처음으로 고객들은 씽큐 앱의 음성인식 기능을 사용해 가전제품을 제어하는 것만 아니라 제품 상태를 파악하고 기능이나 고장 여부를 물어보고 생활 팁까지 얻을 수 있다. 예를 들어, 고객들이 씽큐 앱에 제품을 등록한 다음 앱 화면의 오른쪽 상단에 있는 마이크를 누르고 “에어컨 켜줘” 혹은 “공기청정기 바람 강풍으로 바꿔줘”라고 말하면앱이 음성을 인식해 제품을 작동시키며, “제습기 현재 습도 알려줄래?”, “남은 세탁 시간 알려줘” 등 제품의 상태를 물어보면 “실내 습도는 50% 입니다”, “10분 남았습니다” 등으로 답해준다.

  한편, LG전자는 국내를 시작으로 북미, 유럽 등 글로벌 시장에 씽큐 앱의 음성인식을 추가할 예정이며, 가전관리 앱을 보다 강화하는 것에 맞춰 기존에 LG 스마트씽규로 사용하던 앱 명칭을 올해 10월부터는 LG 씽큐 앱으로 변경하기로 했다. LG전자 H&A스마트홈사업담당 류혜정 전무는 “고객들이 가전을 보다 편리하게 사용하며 스마트홈을 경험할 수 있도록 다양한 기능을 제공할 것”이라고 말했다.

## 이미지 분야 인공지능 기술
* 불량 잡는 'AI의 눈'
  
  [원문 보기](https://www.mk.co.kr/news/business/view/2020/03/283522/)

  __*제품공정 자동화 선도하는 코그넥스*__

  전통 제조기업들에 AI 기술이 큰 의미를 갖는 영역 중 하나는 단연 생산품의 불량률 개선이다. 기존에 수작업으로 할 수밖에 없었던 품질관리(Quality Control·QC) 과정을 AI 기술을 활용해 불량률을 낮추려는 시도가 가속화하고 있는 이유다. AI로 제품과 부품 결함이나 문제를 감지하는 방식 중 주목받는 기술이 딥러닝을 접목한 머신비전이다. 머신비전이란 컴퓨터 이미지 분석을 기계나 로봇 등에 적용해 품질을 제어함으로써 인간의 시각과 판단 능력을 대체하는 기술이다. 여기에 딥러닝까지 접목하면 카메라 촬영 이미지를 AI로 보다 정밀하고 쉽게 분석할 수 있다. 다양한 산업 현장에서 생산성·품질 향상을 위해 이 기술을 도입하고 있다. 코그넥스는 앞선 기술력으로 머신비전 분야를 선도하고 있는 글로벌 기업이다. 자동화된 제조 과정에서의 부품 검사 및 식별, 결함 감지, 제품 조립 확인, 조립 로봇 유도 등에 사용되는 머신비전 시스템과 소프트웨어, 센서를 개발해 생산하고 있다. 미국 기업이지만 20개 이상 국가에 지사를 두고 있다.

  매사추세츠공과대(MIT) 교수였던 로버트 실만 박사가 대학원생이었던 빌 실버(Bill Silver), 메릴린 매츠(Marilyn Matz)와 함께 초기 머신비전의 상용화를 목적으로 회사를 설립하면서 1981년 코그넥스가 탄생했다. 코그넥스 사명은 인지 전문가를 뜻하는 'Cognition Experts'를 줄여서 붙였다. 코그넥스가 1982년 처음 선보인 비전 시스템인 데이터맨(Dataman)은 여러 제품과 구성 요소에 인쇄된 문자나 숫자, 기호의 품질을 확인하고 보장하기 위해 만들어진 광학적 문자 판독장치(OCR)였다. OCR란 빛을 이용해 문자를 판독하는 장치다. 종이에 인쇄되거나 손으로 쓴 문자, 기호 등에 빛을 비춰 그 반사 광선을 전기 신호로 바꾼 다음 컴퓨터에 입력한다. 코그넥스의 첫 고객은 타자기 제조업체였다. 이들은 타자기 키들의 문자를 읽어냄으로써 올바른 위치에 있는지를 확인하기 위해 데이터맨을 구입했다. 1990년대 코그넥스의 성장동력은 반도체와 전자 제품 제조를 자동화하려는 기업들의 머신비전 수요였다. 이 때문에 전자 분야 기업의 성장과 함께 코그넥스도 꾸준히 성장할 수 있었다. 반도체 제조 분야에 적용되는 시스템은 여전히 코그넥스에 중요한 매출원으로 남아 있지만 제약, 헬스케어 등 신사업 분야와 자동차, 패키징, 항공 우주, 소비재 제조 등 다양한 제조업 분야로도 사업 영역을 확대하는 데 성공했다.

  코그넥스는 1995년 미국의 웨이퍼(IC를 제조하는 출발원료인 실리콘 등 반도체의 얇은 판) 식별 시스템 개발 업체인 아큐멘(Acumen)을 인수하는 것을 시작으로 최근까지도 적극적인 M&A 전략을 펼치고 있는 모습이다. 2016년에는 3D 머신비전 기술 업체인 엔스하페와 아큐센스를 인수했으며, 2017년에는 스위스의 AI 머신비전 기업인 비디(ViDi) 시스템스, 지난해 진행한 한국의 AI 기술 기업 수아랩을 인수하기도 했다. 2015년에는 서페이스 비전(Surface Vision) 사업부와 관련 주요 제품을 미국 아메텍(Ametek)에 160억달러를 받고 매각했다.

  코그넥스에서 생산하는 주요 제품은 2D·3D 머신비전 장치, 산업용 바코드 리더기, 비전(이미지 분석) 소프트웨어 등이다. 모두 다양한 산업계의 부품 검사, 식별 등에 사용되는 것들이다. 이미지 분석 소프트웨어 중 하나인 '비전프로 비디(VisionPro Vidi)'는 향후 주목할 만한 제품으로 꼽힌다. 비전프로 비디는 2018년 공장 자동화 전용으로 개발된 최초의 딥러닝 기반 이미지 분석 소프트웨어다. 자동화 시스템의 정확성과 반복 가능성, 빠른 처리 속도 등 장점을 기반으로 제조업계에서 지금까지 자동화하기 어려웠던 여러 작업도 구현 가능할 전망이다. 비전프로 비디의 핵심 기능인 위치·검사·분류 도구에 딥러닝 기반의 광학식 문자 인식을 추가로 지원해 복잡한 글꼴 학습 훈련을 거치지 않아도 표준 텍스트를 판독할 수 있다는 장점을 갖고 있다. 코그넥스가 이러한 장점을 갖춘 비전프로 비디를 개발할 수 있었던 원동력은 비디 시스템스 인수가 있었기에 가능했다.

  코그넥스는 머신비전을 적용할 타깃 산업 분야를 확장하는 데 역량을 쏟고 있다. 올해는 패키징 담당팀을 신설해 운영하는 등 기존에 집중하지 않았던 물류, 제약, 식음료, 유통, 화장품 등 일반 소비재 분야에서도 성과를 낸다는 계획이다.
  

  ## 자율주행 분야 인공지능 기술
* 현대기아차, AI 기반 '부분 자율주행' 기술 최초 개발
  
  [원문 보기](http://www.aitimes.kr/news/articleView.html?idxno=14500)

  __*인공지능 기반 운전자 맞춤형 자율주행*__

  현대자동차와 기아자동차는 운전자의 주행성향에 맞는 부분 자율주행을 구현하는 기술인 SCC-ML을 세계 최초로 개발했다고 21일 밝혔다. 스마트 크루즈 컨트롤 SCC은 앞차와의 거리를 일정하게 유지해 운전자가 설정한 속도로 자율주행을 해주는 기능으로 ADAS(첨단 운전자보조 시스템)의 주요 기술 중 하나다. SCC-ML은 여기에 AI기술을 더해 운전자의 주행성향을 차가 스스로 학습해 SCC 작동 시 운전자와 거의 흡사한 패턴으로 자율주행을 해준다.

  기존의 SCC는 앞차와의 거리, 가속성 등의 주행패턴을 운전자가 직접 설정해야 했으며 조절되는 단계가 세밀하지 않아 운전성향을 고스란히 반영할 수 없었다. 예를 들어, 동일한 운전자라 하더라도 가속성향이 고속과 중속, 저속 구간에서 각각 다르지만 기존에는 이런 세부적인 설정을 변경할 수 없었다. 이 때문에 SCC가 운전자의 주행성향과 다를 경우 운전자는 이질감을 느끼거나 심할 경우 불안감 때문에 SCC 사용을 꺼리는 경우도 더러 있었다.

  이번 현대차·기아차가 독자 개발한 SCC-ML의 원리는 먼저 전방카메라, 레이더 등의 센서가 다양한 운전상황에서 발생되는 정보를 지속적으로 수집해 ADAS의 두뇌격인 제어컴퓨터로 보낸다. 제어컴퓨터는 입력된 정보로부터 운전자의 주행습관을 추출해 종합적인 주행성향을 파악한다. 이때 인공지능 기술 중 하나인 머신러닝 알고리즘이 적용된다. 주행성향은 크게 보면 앞차와의 거리, 가속성(얼마나 신속하게 가속하는지), 반응성(주행환경에 얼마나 민첩하게 반응하는지) 세 가지로 나눌 수 있으며 거기에 더해 다양한 속도와 주변 차량과의 거리 조건을 모두 고려했다. 예를 들어, 저속으로 시내를 주행할 때는 앞차와의 차간거리를 매우 가깝게 유지하나 고속 주행 시에는 차간거리를 멀게 유지할 수도 있다. SCC-ML은 이러한 다양한 상황을 고려해 총만개 이상의 패턴을 구분함으로써 어떤 운전자의 성향에도 맞출 수 있는 SCC 기술을 개발했다.

  주행성향에 대한 정보는 센서를 통해 계속 업데이트되기 때문에 운전자의 최근 성향을 반영할 수 있다. 또 안전운전을 크게 벗어난 주행성향은 따르지 않도록 설정돼 있어서 신뢰성을 높였다. 또 SCC-ML은 자동 차로 변경 기능을 포함하고 있는 HDA II와 함께 적용돼 자율주행 레벨 2을 넘어선 레벨 2.5 수준을 구현한다. 현대차·기아차는 이 같은 기능을 향후 신차에 선별적으로 적용할 예정이다.

 




